<a href="https://colab.research.google.com/github/RO-AD/waymo-od-motion-pred/blob/main/tutorial/4_dataset-structure/hj-dataset-structure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

tutorial/4_dataset-structure/hj-dataset-structure.ipynb

# Dataset Proto 구조 분석

## 환경세팅

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
VALIDATION_PATH = "/content/drive/MyDrive/waymo-od-dataset/motion_v_1_2_0/uncompressed/scenario/validation/"

In [3]:
%%capture
!pip3 install --upgrade pip
!pip install waymo-open-dataset-tf-2-11-0==1.5.1 # 최신 버전 라이브러리

In [4]:

import math
import os
import uuid
import time

import numpy as np
import tensorflow as tf

from google.protobuf import text_format
from waymo_open_dataset.metrics.ops import py_metrics_ops
from waymo_open_dataset.metrics.python import config_util_py as config_util
from waymo_open_dataset.protos import motion_metrics_pb2

In [5]:
filename = VALIDATION_PATH + 'validation.tfrecord-00000-of-00150'

In [6]:
from waymo_open_dataset.protos import scenario_pb2 

dataset = tf.data.TFRecordDataset(filename, compression_type='')
for data in dataset :
   scenario = scenario_pb2.Scenario()
   scenario.ParseFromString(data.numpy())
   break

In [9]:
scenario.scenario_id

'19a486cd29abd7a7'

## Dataset 구조 분석
waymo 공식 깃허브에 있는 proto 파일을 보며 Dataset 구조 분석 해보기
- [scenario.proto](https://github.com/waymo-research/waymo-open-dataset/blob/master/src/waymo_open_dataset/protos/scenario.proto)
- [map.proto](https://github.com/waymo-research/waymo-open-dataset/blob/master/src/waymo_open_dataset/protos/map.proto)
- [compressed_lidar.proto](https://github.com/waymo-research/waymo-open-dataset/blob/master/src/waymo_open_dataset/protos/compressed_lidar.proto)

```proto
// This proto contains the  Waymo Open Dataset Motion source data.

syntax = "proto2";

package waymo.open_dataset;

import "waymo_open_dataset/protos/compressed_lidar.proto";
import "waymo_open_dataset/protos/map.proto";
```

`compressed_lidar.proto`와 `map.proto`를 import 한다.

```proto
message Scenario {
  reserved 9;

  optional string scenario_id = 5;
  repeated double timestamps_seconds = 1;
  optional int32 current_time_index = 10;
  repeated Track tracks = 2;
  repeated DynamicMapState dynamic_map_states = 7;
  repeated MapFeature map_features = 8;
  optional int32 sdc_track_index = 6;
  repeated int32 objects_of_interest = 4;
  repeated RequiredPrediction tracks_to_predict = 11;
  repeated CompressedFrameLaserData compressed_frame_laser_data = 12;
}
```

하나씩 값을 보며 분석

- scenario_id
```proto
optional string scenario_id = 5;
```
- The unique ID for this scenario.
- `oprional`은 옵션을 의미한다.
- 자료형은 `string`

In [23]:
scenario.scenario_id

'19a486cd29abd7a7'

- timestamps_seconds
```proto
repeated double timestamps_seconds = 1;
```
- 0.0부터 0.1초 간격으로 9초까지 91개의 값을 가지고 있는 리스트
- `repeated`는 리스트를 의미한다.
- 자료형은 `double`

In [24]:
scenario.timestamps_seconds

[0.0, 0.10004, 0.20005, 0.3, 0.40001, 0.49998, 0.59998, 0.69998, 0.79999, 0.89999, 0.99999, 1.09995, 1.19996, 1.29992, 1.39991, 1.49988, 1.59987, 1.69992, 1.79991, 1.89994, 1.99994, 2.09994, 2.19994, 2.29995, 2.39998, 2.49998, 2.59998, 2.69998, 2.79998, 2.89998, 2.99998, 3.10001, 3.20005, 3.30009, 3.40013, 3.50013, 3.60013, 3.70017, 3.80017, 3.90017, 4.00017, 4.10018, 4.20014, 4.30011, 4.40006, 4.50002, 4.59999, 4.69995, 4.79992, 4.89988, 4.99988, 5.09984, 5.19984, 5.29984, 5.39984, 5.49988, 5.59989, 5.69993, 5.79993, 5.89996, 6.0, 6.1, 6.2, 6.30003, 6.40003, 6.5, 6.6, 6.7, 6.8, 6.9, 6.99997, 7.09997, 7.19996, 7.29993, 7.39988, 7.49988, 7.59988, 7.69988, 7.79988, 7.89992, 7.99992, 8.09992, 8.19993, 8.29996, 8.39997, 8.49997, 8.59997, 8.69997, 8.79998, 8.89998, 8.99994]

- current_time_index
```proto
optional int32 current_time_index = 10;
```
- 현재 시간의 인덱스. timestamps_seconds[10]은 약 1.0이다.
- 옵션이고 자료형은 `int32`


In [25]:
scenario.current_time_index

10

- tracks
```proto
repeated Track tracks = 2;
```
- 시나리오의 모든 개체 트랙 리스트로, `len(tracks[i].states)`은 91로 동일하다. 
- `tracks[i].states[j]`는 `timestamps_seconds[j]` 시간에 해당하는 i번째 에이전트의 상태이다.
- 자료형은 `Track`이다. `Track` 형태는 같은 파일 내에 선언되어 있다.

In [30]:
print(len(scenario.tracks))
print(len(scenario.tracks[0].states))
print(scenario.tracks[0].id)
print(scenario.tracks[0].states[2])

11
91
368
center_x: 8382.134765625
center_y: 7209.99072265625
center_z: -13.698562246539266
length: 4.469150066375732
width: 1.9771449565887451
height: 1.4942156076431274
heading: -1.5596644878387451
velocity_x: 0.361328125
velocity_y: -19.1748046875
valid: true



- Track
```proto
message Track {
  enum ObjectType {
    TYPE_UNSET = 0;  // This is an invalid state that indicates an error.
    TYPE_VEHICLE = 1;
    TYPE_PEDESTRIAN = 2;
    TYPE_CYCLIST = 3;
    TYPE_OTHER = 4;
  }

  optional int32 id = 1;  // track 개체의 고유 ID
  optional ObjectType object_type = 2; // track 개체 유형. 위에 선언되어 있는 ObjectType 중 하나
  repeated ObjectState states = 3; // 객체 상태 표시. 3D 경계 상자 및 속도 포함
}
```


In [23]:
[e for e in scenario.tracks[0].__dir__() if e[0] != '_'][:10]

['DESCRIPTOR',
 'id',
 'object_type',
 'states',
 'ObjectType',
 'TYPE_UNSET',
 'TYPE_VEHICLE',
 'TYPE_PEDESTRIAN',
 'TYPE_CYCLIST',
 'TYPE_OTHER']

- ObjectState
```proto
message ObjectState {
  // 객체 경계 상자의 중심 좌표
  optional double center_x = 2;
  optional double center_y = 3;
  optional double center_z = 4;

  // 미터 단위의 경계 상자 치수
  optional float length = 5;
  optional float width = 6;
  optional float height = 7;

  // X축(Z축을 중심으로 한 오른쪽 시스템)에서 시계 반대 방향으로 경계 상자의 전방 방향(상자의 중심에서 전면 상자 세그먼트의 중간까지의 벡터)의 yaw 각도(라디안).
  // 이 각도는 [-pi, pi)로 정규화된다.
  optional float heading = 8;

  // 속도 벡터. 이 벡터 방향은 경계 상자의 heading과 약간 다를 수 있다.
  optional float velocity_x = 9;
  optional float velocity_y = 10;

  // 상태 데이터가 잘못되었거나 누락된 경우 False
  optional bool valid = 11;
}
```
- `Track`에서 선언된 `state`의 자료형으로, `scenario.proto`에 있다.


In [25]:
scenario.tracks[0].states[0]

center_x: 8382.083984375
center_y: 7213.89013671875
center_z: -13.732300758361816
length: 4.414564609527588
width: 1.943734884262085
height: 1.4712934494018555
heading: -1.5578702688217163
velocity_x: 0.146484375
velocity_y: -19.8193359375
valid: true

- dynamic_map_states
```proto
repeated DynamicMapState dynamic_map_states = 7;
```
- 시나리오의 동적 맵 상태 리스트 (예: 교통 신호 상태)
- 이 필드의 길이는 `timestamp_seconds`와 동일하다.
- `dynamic_map_states[i]`는 `timestamps_seconds[i]` 시 동적 맵 상태를 인덱싱한다.
- `DynamicMapState` 자료형이고, 해당 자료형은 `scenario.proto`에 선언되어 있다.

- DynamicMapState
```proto
message DynamicMapState {
  // The traffic signal states for all observed signals at this time step.
  repeated TrafficSignalLaneState lane_states = 1;
}
```
- `TrafficSignalLaneState` 자료형의 리스트이다.
- `TrafficSignalLaneState`에 대한 선언은 `map.proto`에 있다.



In [28]:
scenario.dynamic_map_states[0].__dir__()[4]

'lane_states'

- TrafficSignalLaneState
```proto
message TrafficSignalLaneState {
  // 이 트래픽 신호 상태에 의해 제어되는 차선에 해당하는 MapFeature의 ID
  optional int64 lane = 1;

  enum State {
    LANE_STATE_UNKNOWN = 0;

    // 화살표가 있는 신호의 상태
    LANE_STATE_ARROW_STOP = 1;
    LANE_STATE_ARROW_CAUTION = 2;
    LANE_STATE_ARROW_GO = 3;

    // 표준 왕복 교통 신호
    LANE_STATE_STOP = 4;
    LANE_STATE_CAUTION = 5;
    LANE_STATE_GO = 6;

    // 깜빡이는 불빛 신호
    LANE_STATE_FLASHING_STOP = 7;
    LANE_STATE_FLASHING_CAUTION = 8;
  }

  // 트래픽 신호의 상태
  // 위 State에 있는 값 중 하나
  optional State state = 2;

  // 교통 신호에 의해 제어되는 차선의 정지 지점
  // 신호가 정지 상태일 때 동적 물체가 정지해야 하는 지점
  optional MapPoint stop_point = 3;
}
```
- `map.proto`에 선언되어 있다.
- `stop_point`의 자료형인 `MapPoint`도 `map.proto`에 선언되어 있다.
<br><br>
- MapPoint
```proto
message MapPoint {
  // 미터 단위의 위치. origin은 임의의 위치이다.
  optional double x = 1;
  optional double y = 2;
  optional double z = 3;
}
```

In [29]:
scenario.dynamic_map_states[0].lane_states[0]

lane: 120
state: LANE_STATE_GO
stop_point {
  x: 8391.668352965295
  y: 7316.195065592189
  z: -14.848865196389456
}

- map_features
```proto
repeated MapFeature map_features = 8;
```
- 시나리오의 정적 맵 기능 리스트 집합
- 자료형은 `MapFeature`이고, 이 자료형은 `map.proto`에 선언되어 있다.

- MapFeature
```proto
message MapFeature {
  // feature의 고유 ID
  optional int64 id = 1;

  // Type specific data.
  oneof feature_data {
    LaneCenter lane = 3;
    RoadLine road_line = 4;
    RoadEdge road_edge = 5;
    StopSign stop_sign = 7;
    Crosswalk crosswalk = 8;
    SpeedBump speed_bump = 9;
    Driveway driveway = 10;
  }
}
```
- `oneof`는 선언된 필드 중 하나만 사용할 수 있도록 하는 구조체
- `feature_data` 필드 종류
  - LaneCenter: 차선 중앙의 위치를 나타내는 데이터 타입
  - RoadLine: 도로 경계선을 나타내는 데이터 타입
  - RoadEdge: 도로 가장자리를 나타내는 데이터 타입
  - StopSign: 정지 신호를 나타내는 데이터 타입
  - Crosswalk: 횡단보도를 나타내는 데이터 타입
  - SpeedBump: 속도 저하털을 나타내는 데이터 타입
  - Driveway: 차량 출입구를 나타내는 데이터 타입
- `feature_data` 중 어떤 필드인지 알기 위해서는 아래처럼 작성하면 된다.

In [41]:
scenario.map_features[0].WhichOneof('feature_data')

'road_edge'

In [50]:
feature = scenario.map_features[0]

if feature.HasField('lane'):
    feature_data = feature.lane
    print('lane')
elif feature.HasField('road_line'):
    feature_data = feature.road_line
    print('road_line')
elif feature.HasField('road_edge'):
    feature_data = feature.road_edge
    print('road_edge')
elif feature.HasField('stop_sign'):
    feature_data = feature.stop_sign
    print('stop_sign')
elif feature.HasField('crosswalk'):
    feature_data = feature.crosswalk
    print('crosswalk')
elif feature.HasField('speed_bump'):
    feature_data = feature.speed_bump
    print('speed_bump')
elif feature.HasField('driveway'):
    feature_data = feature.driveway
    print('driveway')
else:
    print("No feature data found.")

road_edge


- LaneCenter
```proto
message LaneCenter {
  // 차선 제한 속도
  optional double speed_limit_mph = 1;

  // Type of this lane
  enum LaneType {
    TYPE_UNDEFINED = 0;
    TYPE_FREEWAY = 1;
    TYPE_SURFACE_STREET = 2;
    TYPE_BIKE_LANE = 3;
  }
  optional LaneType type = 2;

  // 차선이 다른 차선 두 개 사이를 보간(두 값 사이에 위치한 값을 추정)하는 경우 True
  optional bool interpolating = 3;

  // 차선의 폴리선(연속된 점 사이에 세그먼트가 정의된 점 리스트) 데이터
  repeated MapPoint polyline = 8;

  // 차선 객체에서 해당 차선으로 진입할 수 있는 차선 ID 리스트
  // packed 형태로 저장되어 있어서 하나의 varint로 여러 개의 entry_lanes를 인코딩
  repeated int64 entry_lanes = 9 [packed = true];

  // 이 차선에서 나갈 수 있는 차선의 ID 리스트
  repeated int64 exit_lanes = 10 [packed = true];

  // BoundarySegment에 대해서는 다음 블록에서 설명
  repeated BoundarySegment left_boundaries = 13;
  repeated BoundarySegment right_boundaries = 14;

  // LaneNeighbor에 대해서는 다음 블록에서 설명
  repeated LaneNeighbor left_neighbors = 11;
  repeated LaneNeighbor right_neighbors = 12;
}
```

- BoundarySegment
```proto
// 지정된 인접 경계가 있는 차선의 세그먼트
message BoundarySegment {
  // 이 차선 경계가 시작되는 차선의 폴리라인에 대한 인덱스
  optional int32 lane_start_index = 1;

  // 이 차선 경계가 끝나는 차선의 폴리라인에 대한 인덱스
  optional int32 lane_end_index = 2;

  // 경계에 대한 지도 feature의 인접 경계 feature ID
  // 이 기능은 RoadLine 기능 또는 RoadEdge 기능일 수 있다.
  optional int64 boundary_feature_id = 3;

  // 인접 경계 유형
  // 경계가 도로 선이 아닌 도로 가장자리인 경우 이 값은 TYPE_UNKNOWN으로 설정
  optional RoadLine.RoadLineType boundary_type = 4;
}
```

- LaneNeighbor
```proto
message LaneNeighbor {
  // 인접 차선의 feature ID
  optional int64 feature_id = 1;

  // 현재 차선 내에서 이웃 차선이 인접한 위치의 시작 인덱스
  optional int32 self_start_index = 2;
  // 현재 차선 내에서 이웃 차선이 인접한 위치의 끝 인덱스
  optional int32 self_end_index = 3;

  // 인접한 차선 내에서 현재 차선이 인접한 위치의 시작 인덱스
  optional int32 neighbor_start_index = 4;
  // 인접한 차선 내에서 현재 차선이 인접한 위치의 끝 인덱스
  optional int32 neighbor_end_index = 5;

  // 현재 차선(차도)과 이웃 차선(차도) 사이의 경계를 정의하는 차선 경계 세그먼트의 리스트
  // 각각의 항목은 경계 유형과 경계가 시작되는 인덱스 및 끝나는 인덱스
  repeated BoundarySegment boundaries = 6;
}
```

- RoadLine
```proto
message RoadLine {
  // Type of this road line
  enum RoadLineType {
    TYPE_UNKNOWN = 0;
    TYPE_BROKEN_SINGLE_WHITE = 1;
    TYPE_SOLID_SINGLE_WHITE = 2;
    TYPE_SOLID_DOUBLE_WHITE = 3;
    TYPE_BROKEN_SINGLE_YELLOW = 4;
    TYPE_BROKEN_DOUBLE_YELLOW = 5;
    TYPE_SOLID_SINGLE_YELLOW = 6;
    TYPE_SOLID_DOUBLE_YELLOW = 7;
    TYPE_PASSING_DOUBLE_YELLOW = 8;
  }
  
  // 위에 선언한 RoadLineType 중 하나
  optional RoadLineType type = 1;

  // 도로 가장자리를 정의하는 폴리선
  // 폴리선은 연속된 점 사이에 세그먼트가 정의된 점 리스트
  // MapPoint는 위에서 정리함
  repeated MapPoint polyline = 2;
}
```

- RoadEdge
```proto
message RoadEdge {
  enum RoadEdgeType {
    TYPE_UNKNOWN = 0;
    TYPE_ROAD_EDGE_BOUNDARY = 1;  // 도로를 외부에서 분리하는 물리적인 도로 가장자리
    TYPE_ROAD_EDGE_MEDIAN = 2;  // 자동차와 다른 교통을 구분하는 물리적인 도로 가장자리
  }
  optional RoadEdgeType type = 1;  // 도로 가장자리
  repeated MapPoint polyline = 2;  
}
```

- StopSign
```proto
message StopSign {
  // 이 정지 신호로 제어되는 차선의 특징 ID
  repeated int64 lane = 1;

  // 정지 신호 위치
  optional MapPoint position = 2;
}
```

- Crosswalk
```proto
message Crosswalk {
  // 횡단보도의 윤곽선을 정의하는 닫힌 다각형
  // 세그먼트가 첫 번째 점과 마지막 점 사이에 존재
  repeated MapPoint polygon = 1;
}
```

- SpeedBump
```proto
message SpeedBump {
  // 과속방지턱의 윤곽을 정의하는 닫힌 다각형
  // 세그먼트가 첫 번째 점과 마지막 점 사이에 존재
  repeated MapPoint polygon = 1;
}
```

- Driveway
```proto
message Driveway {
  // 차량 진입로 영역의 윤곽선을 정의하는 닫힌 다각형
  // 세그먼트가 첫 번째 점과 마지막 점 사이에 존재
  repeated MapPoint polygon = 1;
}
```

In [52]:
scenario.map_features[0].road_edge.polyline[0]

x: 8292.619348114988
y: 7179.778677513115
z: -14.258546045464358

- sdc_track_index
```proto
optional int32 sdc_track_index = 6;
```
- sdc : Self-Driving Car
- 맵 정보를 촬영하는 자율주행 차량 개체의 track 리스트에 해당하는 인덱스 (옵션)
- 보행자, 자전거, 자동차 등의 에이전트를 포함한 track 리스트 중 해당 인덱스는 무시

In [43]:
scenario.sdc_track_index

10

- objects_of_interest
```proto
repeated int32 objects_of_interest = 4;
```
- 상호작용(interactive) 동작이 감지된 scene의 객체(Object) ID 목록 리스트
- 이 필드에 포함된 ID는 tracks 필드의 ID에 해당되며, tracks 필드의 객체와 일치한다
- 예를 들어, 자율주행 자동차가 도로를 따라 이동하는 동안, 
- 이 필드는 차량 주위에 위치한 보행자나 자전거 등과 같은 상호작용 객체 ID를 포함할 수 있다
- 이러한 객체들은 자율주행 자동차와의 상호작용을 감지하는 데 사용될 수 있다.

In [46]:
scenario.objects_of_interest

[]

- tracks_to_predict
```proto
repeated RequiredPrediction tracks_to_predict = 11;
```
- 예측을 생성하려는 추적 대상 객체 ID의 목록 리스트
- 자료형은 `RequiredPrediction`로 `scenario.proto`에 정의되어 있다.
- 해당 필드는 추적 대상 객체를 학습하는 데 사용된다.
- tracks[tracks_to_predict.track_index].states[15::5]의 center_x, center_y를 예측해야 한다.
- 즉, future timestamp인 1.5초부터 0.5초 간격으로 states의 center_x, center_y를 예측해야 한다. 

- RequiredPrediction
```proto
// 시나리오에 대해 예측해야 하는 개체
message RequiredPrediction {
  // 주어진 트랙을 예측하기 위한 난이도
  enum DifficultyLevel {
    NONE = 0;
    LEVEL_1 = 1;
    LEVEL_2 = 2;
  }

  // 예측할 객체에 대한 시나리오 tracks 필드의 인덱스
  optional int32 track_index = 1;

  // 위에서 선언한 난이도 구조체
  optional DifficultyLevel difficulty = 2;
}
```

In [53]:
scenario.tracks_to_predict

[track_index: 2
difficulty: LEVEL_1
]

- compressed_frame_laser_data
```proto
repeated CompressedFrameLaserData compressed_frame_laser_data = 12;
```
- time step에서의 Lidar 데이터를 담고 있다
- compressed_frame_laser_data[i]는 timestamps_seconds[i]에서의 상태를 나타낸다
- 이 필드는 현재 시간 인덱스보다 작거나 같은 모든 시간 단계에 대한 데이터를 포함한다


In [48]:
scenario.compressed_frame_laser_data

[]

`compressed_lidar.proto` 구조에 대해서는 분석하지 않았다.